<a href="https://colab.research.google.com/github/LollipopGB/EA_TechnicalTest_DanielGarcia/blob/master/MBERT_Word_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Embeddings with TensorFlow 2.0
With the new release of TensorFlow, this Notebook aims to show a simple use of the BERT model.
- See BERT on paper: https://arxiv.org/pdf/1810.04805.pdf
- See BERT on GitHub: https://github.com/google-research/bert
- See BERT on TensorHub: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1
- See 'old' use of BERT for comparison: https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow==2.0
!pip install tensorflow_hub
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 86.3MB 72kB/s 
     |████████████████████████████████| 450kB 43.9MB/s 
     |████████████████████████████████| 3.8MB 38.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=40e4d846155864683e6fa9c8659df56db3ff72adf9d2b00714756a1bdddbc4b7
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: t

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

TF version:  2.0.0
Hub version:  0.9.0.dev


## Import modules

In [3]:
import tensorflow_hub as hub
import tensorflow as tf
import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math
import pandas as pd
import string

In [4]:
path = '/content/drive/My Drive/ea_corpora_no_nan.csv'
df = pd.read_csv(path, sep=',', header=0)
max_seq_length = 128

## Building the model

Building model using tf.keras and hub. from sentences to embeddings.

Inputs:
 - input token ids (tokenizer converts tokens using vocab file)
 - input masks (1 for useful tokens, 0 for padding)
 - segment ids (for 2 text training: 0 for the first one, 1 for the second one)

Outputs:
 - pooled_output of shape `[batch_size, 768]` with representations for the entire input sequences
 - sequence_output of shape `[batch_size, max_seq_length, 768]` with representations for each input token (in context)

In [5]:
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2",
                            trainable=False)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

In [6]:
model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

## Example of generating an embedding

First, we preprocessed the sentence following the BERT methodology. Then, we generate the ids, mask and segments with the tokenizer.

In [7]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [42]:
s = "This nice, sentence."

Tokenizing the sentence

In [43]:
stokens = tokenizer.tokenize(s)

Adding separator tokens according to the paper

In [44]:
stokens = ["[CLS]"] + stokens + ["[SEP]"]

Get the model inputs from the tokens

In [45]:
input_ids = get_ids(stokens, tokenizer, max_seq_length)
input_masks = get_masks(stokens, max_seq_length)
input_segments = get_segments(stokens, max_seq_length)

In [46]:
print(stokens)
print(input_ids)
print(input_masks)
print(input_segments)

['[CLS]', 'This', 'nic', '##e', ',', 'sentence', '.', '[SEP]']
[101, 10747, 46267, 10112, 117, 49219, 119, 102]
[1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0]


Generate Embeddings using the pretrained model

In [ ]:
pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])

In [ ]:
pool_embs

array([[ 0.36660245, -0.12976474,  0.36954308, -0.30230048, -0.12688649,
         0.4480722 ,  0.34225437,  0.4513771 , -0.6195292 ,  0.40002117,
        -0.01895287, -0.32149178, -0.4154251 , -0.235881  ,  0.04574116,
        -0.35588464,  0.86814296, -0.07049157,  0.18631254, -0.41917616,
        -0.9999432 , -0.3689946 , -0.38849938, -0.28183067, -0.6432516 ,
         0.27807367, -0.31455982,  0.43793795,  0.34565958, -0.4184718 ,
         0.21124595, -0.99993414,  0.6645689 ,  0.78652436,  0.4669484 ,
        -0.37837976,  0.21676782,  0.3175907 ,  0.27327144, -0.37613043,
        -0.3541453 , -0.07825275, -0.23527975,  0.29091343, -0.28101763,
        -0.44499934, -0.28636032,  0.37217   , -0.5047371 ,  0.03486316,
        -0.03900278,  0.4357047 ,  0.6393256 ,  0.40566805,  0.3666655 ,
         0.16644214,  0.3175498 ,  0.28390923,  0.49938804, -0.3719989 ,
         0.0089951 ,  0.4893989 ,  0.33697465, -0.20178486, -0.17288496,
        -0.41409138,  0.2859593 , -0.17363161,  0.7

## Generate the dataset

Define auxiliar methods to generate the batches to obtain the word embeddings per document.

At the beginning, the documents were trim to 1000 words (the 3rd quartile of number of documents per total size) to generate small number of batches per document without losing too much information. However, it was going to take about 17 hours to generate all the embeddings, so just to show the whole process and reduce computational time as que are not seeking the best performance and quality, the texts are greatly reduced to 100 words. The execution time was still 6 hours. So, we are going to reduce the dataset length, **just to show the process with the classifier**.

In [8]:
def reduce_text(text):
  if len(text.split()) > 100:
    return " ".join(text.split()[:100])
  else:
    return text

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def generate_batches(text, size):
  chunks = [text[x:x+size] for x in range(0, len(text), size)]
  for c in chunks:
    c.insert(0, '[CLS]')
    c.append('[SEP]')
  return chunks

def generate_all(row):
  data_prepared = []
  for i in row:
    input_ids = get_ids(i, tokenizer, max_seq_length)
    input_masks = get_masks(i, max_seq_length)
    input_segments = get_segments(i, max_seq_length)
    data_prepared.append([input_ids, input_masks, input_segments])
  return data_prepared

def get_embeddings(batches):
  embedding = []
  for batch in batches:
    pool_embs, all_embs = model.predict([[batch[0]], [batch[1]], [batch[2]]])
    embedding.append(pool_embs)
  return embedding

In [32]:
df_reduced = df.iloc[range(0,len(df),3)]
df_reduced = df_reduced.reset_index()

In [34]:
df_reduced.shape

(7847, 8)

In [35]:
df_reduced['text_reduced'] = df_reduced['text'].apply(lambda x: reduce_text(x))

In [36]:
df_reduced['text_tokens'] = df_reduced['text_reduced'].apply(lambda x: tokenizer.tokenize(x))

In [37]:
df_reduced['text_batches'] = df_reduced['text_tokens'].apply(lambda x: generate_batches(x, 100))

In [38]:
df_reduced['basic_set'] = df_reduced['text_batches'].apply(lambda x: generate_all(x))

In [39]:
df_reduced.head()

,index,text,category,language,text_reduced,text_tokens,text_batches,basic_set
0,0,"i read this book because in my town, everyone ...",APR,en,"i read this book because in my town, everyone ...","[i, read, this, book, because, in, my, town, ,...","[[[CLS], i, read, this, book, because, in, my,...","[[[101, 177, 24944, 10531, 12748, 12373, 10106..."
1,3,milady has found a good vein: anita blake. bas...,APR,en,milady has found a good vein: anita blake. bas...,"[mil, ##ady, has, found, a, good, vei, ##n, :,...","[[[CLS], mil, ##ady, has, found, a, good, vei,...","[[[101, 15033, 51210, 10393, 11823, 169, 15198..."
2,6,"well, frankly, the first 3 volumes of the new ...",APR,en,"well, frankly, the first 3 volumes of the new ...","[well, ,, fra, ##nk, ##ly, ,, the, first, 3, v...","[[[CLS], well, ,, fra, ##nk, ##ly, ,, the, fir...","[[[101, 11206, 117, 10628, 17761, 10454, 117, ..."
3,9,it is a deafening silence olivier delorme brea...,APR,en,it is a deafening silence olivier delorme brea...,"[it, is, a, dea, ##fen, ##ing, silence, oli, #...","[[[CLS], it, is, a, dea, ##fen, ##ing, silence...","[[[101, 10271, 10124, 169, 42492, 15559, 10230..."
4,12,"i really like if it was true, and i felt faint...",APR,en,"i really like if it was true, and i felt faint...","[i, really, like, if, it, was, true, ,, and, i...","[[[CLS], i, really, like, if, it, was, true, ,...","[[[101, 177, 30181, 11850, 12277, 10271, 10134..."


In [17]:
from tqdm import tqdm

In [40]:
tqdm.pandas()
df_reduced['embeddings'] = df_reduced['basic_set'].progress_apply(lambda x: get_embeddings(x))

Se han truncado las últimas 5000 líneas del flujo de salida.
100%|██████████| 7847/7847 [2:55:01<00:00,  1.34s/it]


In [70]:
df_reduced.head()

,index,text,category,language,text_reduced,text_tokens,text_batches,basic_set,embeddings
0,0,"i read this book because in my town, everyone ...",APR,en,"i read this book because in my town, everyone ...","[i, read, this, book, because, in, my, town, ,...","[[[CLS], i, read, this, book, because, in, my,...","[[[101, 177, 24944, 10531, 12748, 12373, 10106...","[[[0.21412727, -0.25563636, 0.3922883, -0.1099..."
1,3,milady has found a good vein: anita blake. bas...,APR,en,milady has found a good vein: anita blake. bas...,"[mil, ##ady, has, found, a, good, vei, ##n, :,...","[[[CLS], mil, ##ady, has, found, a, good, vei,...","[[[101, 15033, 51210, 10393, 11823, 169, 15198...","[[[0.21550941, -0.32545748, 0.33222845, -0.287..."
2,6,"well, frankly, the first 3 volumes of the new ...",APR,en,"well, frankly, the first 3 volumes of the new ...","[well, ,, fra, ##nk, ##ly, ,, the, first, 3, v...","[[[CLS], well, ,, fra, ##nk, ##ly, ,, the, fir...","[[[101, 11206, 117, 10628, 17761, 10454, 117, ...","[[[0.026787607, -0.3051752, 0.072571024, -0.58..."
3,9,it is a deafening silence olivier delorme brea...,APR,en,it is a deafening silence olivier delorme brea...,"[it, is, a, dea, ##fen, ##ing, silence, oli, #...","[[[CLS], it, is, a, dea, ##fen, ##ing, silence...","[[[101, 10271, 10124, 169, 42492, 15559, 10230...","[[[0.16710715, -0.19441627, 0.23410313, -0.311..."
4,12,"i really like if it was true, and i felt faint...",APR,en,"i really like if it was true, and i felt faint...","[i, really, like, if, it, was, true, ,, and, i...","[[[CLS], i, really, like, if, it, was, true, ,...","[[[101, 177, 30181, 11850, 12277, 10271, 10134...","[[[0.3504453, -0.23025343, 0.31962156, -0.2193..."


In [90]:
data = []

for i in range(len(df_reduced)):

  new_row = [item for sublist in df_reduced.loc[6]['embeddings'] for item in sublist.tolist()[0]]

  new_row.insert(0, df_reduced.loc[i]['category'])
  new_row.insert(0, df_reduced.loc[i]['language'])
  data.append(new_row)

In [91]:
df_flatten = pd.DataFrame(data)

In [92]:
df_flatten.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769
0,en,APR,0.251807,-0.298893,0.37124,-0.256563,-0.12887,0.484728,0.269987,0.372555,-0.503706,0.415584,-0.089458,-0.267181,-0.3462,-0.167397,0.173834,-0.328256,0.791704,0.050997,0.114166,-0.265748,-0.999975,-0.188566,-0.372232,-0.207628,-0.43567,0.231051,-0.167573,0.251028,0.196864,-0.160173,0.289802,-0.999971,0.709407,0.690658,0.353739,-0.041841,0.128136,0.247878,...,0.273952,-0.057707,0.351565,-0.160594,0.323168,0.235397,-0.272889,0.149716,-0.300301,0.278905,-0.25642,-0.010798,-0.996473,-0.320502,0.196619,0.400121,0.358131,-0.325586,-0.320847,-0.322447,-0.254568,0.297849,0.286724,0.390469,0.278065,0.142473,-0.251893,-0.065769,0.839429,-0.325394,0.020349,0.485651,0.188433,0.891282,0.398408,0.30282,0.259855,-0.309314,0.272197,0.236462
1,en,APR,0.251807,-0.298893,0.37124,-0.256563,-0.12887,0.484728,0.269987,0.372555,-0.503706,0.415584,-0.089458,-0.267181,-0.3462,-0.167397,0.173834,-0.328256,0.791704,0.050997,0.114166,-0.265748,-0.999975,-0.188566,-0.372232,-0.207628,-0.43567,0.231051,-0.167573,0.251028,0.196864,-0.160173,0.289802,-0.999971,0.709407,0.690658,0.353739,-0.041841,0.128136,0.247878,...,0.273952,-0.057707,0.351565,-0.160594,0.323168,0.235397,-0.272889,0.149716,-0.300301,0.278905,-0.25642,-0.010798,-0.996473,-0.320502,0.196619,0.400121,0.358131,-0.325586,-0.320847,-0.322447,-0.254568,0.297849,0.286724,0.390469,0.278065,0.142473,-0.251893,-0.065769,0.839429,-0.325394,0.020349,0.485651,0.188433,0.891282,0.398408,0.30282,0.259855,-0.309314,0.272197,0.236462
2,en,APR,0.251807,-0.298893,0.37124,-0.256563,-0.12887,0.484728,0.269987,0.372555,-0.503706,0.415584,-0.089458,-0.267181,-0.3462,-0.167397,0.173834,-0.328256,0.791704,0.050997,0.114166,-0.265748,-0.999975,-0.188566,-0.372232,-0.207628,-0.43567,0.231051,-0.167573,0.251028,0.196864,-0.160173,0.289802,-0.999971,0.709407,0.690658,0.353739,-0.041841,0.128136,0.247878,...,0.273952,-0.057707,0.351565,-0.160594,0.323168,0.235397,-0.272889,0.149716,-0.300301,0.278905,-0.25642,-0.010798,-0.996473,-0.320502,0.196619,0.400121,0.358131,-0.325586,-0.320847,-0.322447,-0.254568,0.297849,0.286724,0.390469,0.278065,0.142473,-0.251893,-0.065769,0.839429,-0.325394,0.020349,0.485651,0.188433,0.891282,0.398408,0.30282,0.259855,-0.309314,0.272197,0.236462
3,en,APR,0.251807,-0.298893,0.37124,-0.256563,-0.12887,0.484728,0.269987,0.372555,-0.503706,0.415584,-0.089458,-0.267181,-0.3462,-0.167397,0.173834,-0.328256,0.791704,0.050997,0.114166,-0.265748,-0.999975,-0.188566,-0.372232,-0.207628,-0.43567,0.231051,-0.167573,0.251028,0.196864,-0.160173,0.289802,-0.999971,0.709407,0.690658,0.353739,-0.041841,0.128136,0.247878,...,0.273952,-0.057707,0.351565,-0.160594,0.323168,0.235397,-0.272889,0.149716,-0.300301,0.278905,-0.25642,-0.010798,-0.996473,-0.320502,0.196619,0.400121,0.358131,-0.325586,-0.320847,-0.322447,-0.254568,0.297849,0.286724,0.390469,0.278065,0.142473,-0.251893,-0.065769,0.839429,-0.325394,0.020349,0.485651,0.188433,0.891282,0.398408,0.30282,0.259855,-0.309314,0.272197,0.236462
4,en,APR,0.251807,-0.298893,0.37124,-0.256563,-0.12887,0.484728,0.269987,0.372555,-0.503706,0.415584,-0.089458,-0.267181,-0.3462,-0.167397,0.173834,-0.328256,0.791704,0.050997,0.114166,-0.265748,-0.999975,-0.188566,-0.372232,-0.207628,-0.43567,0.231051,-0.167573,0.251028,0.196864,-0.160173,0.289802,-0.999971,0.709407,0.690658,0.353739,-0.041841,0.128136,0.247878,...,0.273952,-0.057707,0.351565,-0.160594,0.323168,0.235397,-0.272889,0.149716,-0.300301,0.278905,-0.25642,-0.010798,-0.996473,-0.320502,0.196619,0.400121,0.358131,-0.325586,-0.320847,-0.322447,-0.254568,0.297849,0.286724,0.390469,0.278065,0.142473,-0.251893,-0.065769,0.839429,-0.325394,0.020349,0.485651,0.188433,0.891282,0.398408,0.30282,0.259855,-0.309314,0.272197,0.236462


In [93]:
df_flatten = df_flatten.rename(columns={0: 'language', 1: 'category'})

In [94]:
df_flatten.head()

,language,category,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769
0,en,APR,0.251807,-0.298893,0.37124,-0.256563,-0.12887,0.484728,0.269987,0.372555,-0.503706,0.415584,-0.089458,-0.267181,-0.3462,-0.167397,0.173834,-0.328256,0.791704,0.050997,0.114166,-0.265748,-0.999975,-0.188566,-0.372232,-0.207628,-0.43567,0.231051,-0.167573,0.251028,0.196864,-0.160173,0.289802,-0.999971,0.709407,0.690658,0.353739,-0.041841,0.128136,0.247878,...,0.273952,-0.057707,0.351565,-0.160594,0.323168,0.235397,-0.272889,0.149716,-0.300301,0.278905,-0.25642,-0.010798,-0.996473,-0.320502,0.196619,0.400121,0.358131,-0.325586,-0.320847,-0.322447,-0.254568,0.297849,0.286724,0.390469,0.278065,0.142473,-0.251893,-0.065769,0.839429,-0.325394,0.020349,0.485651,0.188433,0.891282,0.398408,0.30282,0.259855,-0.309314,0.272197,0.236462
1,en,APR,0.251807,-0.298893,0.37124,-0.256563,-0.12887,0.484728,0.269987,0.372555,-0.503706,0.415584,-0.089458,-0.267181,-0.3462,-0.167397,0.173834,-0.328256,0.791704,0.050997,0.114166,-0.265748,-0.999975,-0.188566,-0.372232,-0.207628,-0.43567,0.231051,-0.167573,0.251028,0.196864,-0.160173,0.289802,-0.999971,0.709407,0.690658,0.353739,-0.041841,0.128136,0.247878,...,0.273952,-0.057707,0.351565,-0.160594,0.323168,0.235397,-0.272889,0.149716,-0.300301,0.278905,-0.25642,-0.010798,-0.996473,-0.320502,0.196619,0.400121,0.358131,-0.325586,-0.320847,-0.322447,-0.254568,0.297849,0.286724,0.390469,0.278065,0.142473,-0.251893,-0.065769,0.839429,-0.325394,0.020349,0.485651,0.188433,0.891282,0.398408,0.30282,0.259855,-0.309314,0.272197,0.236462
2,en,APR,0.251807,-0.298893,0.37124,-0.256563,-0.12887,0.484728,0.269987,0.372555,-0.503706,0.415584,-0.089458,-0.267181,-0.3462,-0.167397,0.173834,-0.328256,0.791704,0.050997,0.114166,-0.265748,-0.999975,-0.188566,-0.372232,-0.207628,-0.43567,0.231051,-0.167573,0.251028,0.196864,-0.160173,0.289802,-0.999971,0.709407,0.690658,0.353739,-0.041841,0.128136,0.247878,...,0.273952,-0.057707,0.351565,-0.160594,0.323168,0.235397,-0.272889,0.149716,-0.300301,0.278905,-0.25642,-0.010798,-0.996473,-0.320502,0.196619,0.400121,0.358131,-0.325586,-0.320847,-0.322447,-0.254568,0.297849,0.286724,0.390469,0.278065,0.142473,-0.251893,-0.065769,0.839429,-0.325394,0.020349,0.485651,0.188433,0.891282,0.398408,0.30282,0.259855,-0.309314,0.272197,0.236462
3,en,APR,0.251807,-0.298893,0.37124,-0.256563,-0.12887,0.484728,0.269987,0.372555,-0.503706,0.415584,-0.089458,-0.267181,-0.3462,-0.167397,0.173834,-0.328256,0.791704,0.050997,0.114166,-0.265748,-0.999975,-0.188566,-0.372232,-0.207628,-0.43567,0.231051,-0.167573,0.251028,0.196864,-0.160173,0.289802,-0.999971,0.709407,0.690658,0.353739,-0.041841,0.128136,0.247878,...,0.273952,-0.057707,0.351565,-0.160594,0.323168,0.235397,-0.272889,0.149716,-0.300301,0.278905,-0.25642,-0.010798,-0.996473,-0.320502,0.196619,0.400121,0.358131,-0.325586,-0.320847,-0.322447,-0.254568,0.297849,0.286724,0.390469,0.278065,0.142473,-0.251893,-0.065769,0.839429,-0.325394,0.020349,0.485651,0.188433,0.891282,0.398408,0.30282,0.259855,-0.309314,0.272197,0.236462
4,en,APR,0.251807,-0.298893,0.37124,-0.256563,-0.12887,0.484728,0.269987,0.372555,-0.503706,0.415584,-0.089458,-0.267181,-0.3462,-0.167397,0.173834,-0.328256,0.791704,0.050997,0.114166,-0.265748,-0.999975,-0.188566,-0.372232,-0.207628,-0.43567,0.231051,-0.167573,0.251028,0.196864,-0.160173,0.289802,-0.999971,0.709407,0.690658,0.353739,-0.041841,0.128136,0.247878,...,0.273952,-0.057707,0.351565,-0.160594,0.323168,0.235397,-0.272889,0.149716,-0.300301,0.278905,-0.25642,-0.010798,-0.996473,-0.320502,0.196619,0.400121,0.358131,-0.325586,-0.320847,-0.322447,-0.254568,0.297849,0.286724,0.390469,0.278065,0.142473,-0.251893,-0.065769,0.839429,-0.325394,0.020349,0.485651,0.188433,0.891282,0.398408,0.30282,0.259855,-0.309314,0.272197,0.236

In [95]:
df_flatten.to_csv('/content/drive/My Drive/ea_embeddings_bert_flatten.csv', index=False, encoding='utf-8') 